In [1]:
from dataset import (
    get_dataset, 
    save_map, 
    save_pred_map_to_tif,
    save_dataset_to_tif,
)
from metrics import (
    get_metrics,
    get_classification_metrics,
    get_feature_importances, 
    get_catboost_importances, 
)
from vis import (
    plot_image,
    plot_mask,
    plot_SCL,
    draw_maps,
)

import matplotlib.pyplot as plt
from rasterio.windows import Window
import pandas as pd
import wandb
import rasterio
import numpy as np
import tifffile as tiff
import wandb
from sklearn.model_selection import train_test_split

%load_ext autoreload
%autoreload 2   

/beegfs/home/alina.smolina/forest-co2/dataset.py:10: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
import rasterio
from rasterio.windows import Window
from shapely.geometry.polygon import Polygon
from shapely import geometry
import tifffile as tiff
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from pathlib import Path
import uuid

from dataset import (
    get_dataset, 
    save_map,
)
from vis import (
    plot_res
)

%load_ext autoreload
%autoreload 2   

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
path_S2B = './data/tile_UTM38N/rasterized/'

In [6]:
with rasterio.open(f'{path_S2B}/age.tif') as src:
    age_map = src.read(1)
    
with rasterio.open(f'{path_S2B}/volume.tif') as src:
    volume_map = src.read(1)
    
with rasterio.open(f'{path_S2B}/type.tif') as src:
    type_map = src.read(1)

In [12]:
def get_stock_map(age_map, volume_map, type_map):
    age = age_map
    
    species = type_map
    
    volume = volume_map
    
    species_age = {
    1: [[0, 40], [40, 100], [100, 120], [120, 1000]],
    2: [[0, 20], [20, 40], [40, 50], [50, 1000]],
    3: [[0, 20], [20, 40], [40, 50], [50, 1000]],
    4: [[0, 20], [20, 30], [30, 40], [40, 1000]],
    5: [[0, 40], [40, 100], [100, 120], [120, 1000]],
              }

    species_coeff = {
        1: [0.409, 0.426, 0.431, 0.436],
        2: [0.528, 0.534, 0.533, 0.528],
        3: [0.45, 0.46, 0.46, 0.46],
        4: [0.403, 0.426, 0.431, 0.444],
        5: [0.452, 0.454, 0.454, 0.449],
                    }

    C_content = {
        1: 0.5,
        2: 0.47,
        3: 0.48,
        4: 0.47, 
        5: 0.5
    }
    
    BCEF_species = {
    1: np.zeros(volume.shape),
    2: np.zeros(volume.shape), 
    3: np.zeros(volume.shape),
    4: np.zeros(volume.shape),
    5: np.zeros(volume.shape),
               }

    tmp_shape = np.ones(volume.shape)

    for sp in BCEF_species.keys():
        for age_ind in range(4):
            specimen_mask = np.where(species == sp, 1, 0) 
            age_condition = (species_age[sp][age_ind][0] * tmp_shape < age) & (age <= species_age[sp][age_ind][1] * tmp_shape)
            BCEF_species[sp] += np.where(age_condition, np.squeeze(specimen_mask * species_coeff[sp][age_ind]), 0)
        BCEF_species[sp] *= C_content[sp]

    carbon_stock = volume * sum(BCEF_species.values())
    carbon_stock = np.where(carbon_stock > 0, carbon_stock, 0)
    
    return carbon_stock

In [13]:
stock_map = get_stock_map(age_map, volume_map, type_map)

In [14]:
stock_map.shape

(9452, 10345)

In [16]:
stock_map = np.where(age_map > 0, stock_map, 0)

In [18]:
with rasterio.open(f'{path_S2B}/age.tif') as src:
    params = src.meta.copy()

with rasterio.open(f'./preds/carbon_stock.tif', 'w', **params) as dst: 
    dst.write(stock_map, indexes=1)